In [1]:
import pandas as pd
from geopy.distance import great_circle

In [3]:
metro = pd.read_csv('moscow_metro.csv')
metro.head()

,Unnamed: 0,station_name,station_lat,station_lng,line_id,line_name,station_id,station_order
0,0,Новокосино,55.745113,37.864052,8,Калининская,8.189,0
1,1,Новогиреево,55.752237,37.814587,8,Калининская,8.880,1
2,2,Перово,55.750980,37.784220,8,Калининская,8.107,2
3,3,Шоссе Энтузиастов,55.758090,37.751703,8,Калининская,8.158,3
4,4,Авиамоторная,55.751933,37.717444,8,Калининская,8.100,4


In [ ]:
cafe = pd.read_csv('cafes_gdf.csv') # датасет с заведением
cafe.head()

,Unnamed: 0,city,object,type,geometry,lat,lon
0,0,Москва,amenity,cafe,POINT (37.5271538 55.6517026),55.651703,37.527154
1,1,Москва,amenity,cafe,POINT (37.6888836 55.7052212),55.705221,37.688884
2,2,Москва,amenity,cafe,POINT (37.6787149 55.7897054),55.789705,37.678715
3,3,Москва,amenity,cafe,POINT (37.7857738 55.6976881),55.697688,37.785774
4,4,Москва,amenity,cafe,POINT (37.5729914 55.6866726),55.686673,37.572991


In [ ]:
# метро в словарь преобразуем
metro_dict = metro.set_index(['station_name'])[['station_lat', 'station_lng', 'line_name']].apply(lambda x: [x['station_lat'], x['station_lng'], x['line_name']], axis=1).to_dict()

In [ ]:
# расстояние
def calculate_distance(lat1, lon1, lat2, lon2):
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    
    distance = great_circle(point1, point2).kilometers
    return distance

# пробегаемся по всем метро
def find_nearest_metro(cafe_lat, cafe_lon, metro_dict):
    min_distance = float('inf')
    nearest_station = None
    
    for station, (station_lat, station_lng, nearest_line) in metro_dict.items():
        distance = calculate_distance(cafe_lat, cafe_lon, station_lat, station_lng)
        if distance < min_distance:
            min_distance = distance
            nearest_station = station
    
    return nearest_station, min_distance, nearest_line

cafe['nearest_metro'], cafe['distance_to_metro'], cafe['nearest_line'] = zip(*cafe.apply(lambda row: find_nearest_metro(row['lat'], row['lon'], metro_dict), axis=1))

cafe

,Unnamed: 0,city,object,type,geometry,lat,lon,nearest_metro,distance_to_metro,nearest_line
0,0,Москва,amenity,cafe,POINT (37.5271538 55.6517026),55.651703,37.527154,Калужская,0.981658,Троицкая
1,1,Москва,amenity,cafe,POINT (37.6888836 55.7052212),55.705221,37.688884,Кожуховская,0.239483,Троицкая
2,2,Москва,amenity,cafe,POINT (37.6787149 55.7897054),55.789705,37.678715,Сокольники,0.156674,Троицкая
3,3,Москва,amenity,cafe,POINT (37.7857738 55.6976881),55.697688,37.785774,Кузьминки,1.654403,Троицкая
4,4,Москва,amenity,cafe,POINT (37.5729914 55.6866726),55.686673,37.572991,Академическая,0.068271,Троицкая
...,...,...,...,...,...,...,...,...,...,...
4577,4577,Москва,amenity,cafe,"POLYGON ((37.627741 55.825732, 37.6277446 55.8...",55.825775,37.627914,Улица Академика Королёва,0.442169,Троицкая
4578,4578,Москва,amenity,cafe,"POLYGON ((37.6270365 55.825999, 37.6270507 55....",55.826050,37.627157,Улица Академика Королёва,0.470197,Троицкая
4579,4579,Москва,amenity,cafe,"POLYGON ((37.6197896 55.8299725, 37.6197904 55...",55.829953,37.619932,Улица Академика Королёва,1.010858,Троицкая
4580,4580,Москва,amenity,cafe,"POLYGON ((37.6478652 55.8206125, 37.6478706 55...",55.820645,37.647926,ВДНХ,0.462312,Троицкая
